In [1]:
import numpy as np
import pandas as pd

In [2]:
!ls

GestureLabelling.ipynb
Playing_Around_with_LSTM.ipynb
Untitled.ipynb
drone_ges_model-nonpipeline.ipynb
drone_pos_model-nonpipeline.h5
drone_pos_model-nonpipeline.ipynb
drone_pos_model.ipynb
drone_pose_data_analysis.ipynb
model_json
pascal_notebook.ipynb


In [3]:
data_dir = "../data/GestureData/"
file_name = "Circle_V01_Pascal_Raw_labels.csv"

In [76]:
df = pd.read_csv(data_dir + file_name)

In [86]:
df

,from,to,label,real_start,real_end,diff
0,20.45,20.70,1,18.70,20.45,-0.25
1,22.40,22.40,1,20.40,22.40,0.00
2,24.35,24.50,1,22.35,24.50,0.15
3,28.55,28.65,1,26.65,28.55,-0.10
4,32.60,32.70,1,30.70,32.60,-0.10
5,36.35,36.65,1,34.65,36.35,-0.30
6,40.55,40.65,1,38.65,40.55,-0.10
7,44.50,44.60,1,42.60,44.50,-0.10
8,48.40,48.50,1,46.50,48.40,-0.10
9,52.50,52.70,1,50.70,52.50,-0.20


In [124]:
def get_to_from(df, symmetric_slack = 0.0, min_slack = 0.1):

    _diff = df.real_end - df.real_start - 2.0
    _indicator = (_diff >= 0).astype("int32")
    
    
    _lower_slack = - _indicator * _diff -\
        (symmetric_slack-min_slack)*(symmetric_slack-min_slack > 0.001)
    _upper_slack = - (1-_indicator) * _diff +\
        (symmetric_slack-min_slack)*(symmetric_slack-min_slack > 0.001)
    
    _min_slack_ind = (_upper_slack - _lower_slack < 0.1 - 0.001)\
                .astype("int32")
    
    _lower_slack = _lower_slack - min_slack/2 * _min_slack_ind
    _upper_slack = _upper_slack + min_slack/2 * _min_slack_ind
    
    df["from"] = df.real_end + _lower_slack
    df["to"] = df.real_end + _upper_slack
       
    _info = pd.DataFrame(
        columns=["diff","indicator","l_slack","u_slack","min_slack_ind"]
    )
    _info["diff"] = _diff
    _info["indicator"] = _indicator
    _info["l_slack"] = np.round(_lower_slack,2) 
    _info["u_slack"] = np.round(_upper_slack,2)
    _info["min_slack_ind"] = _min_slack_ind
    
    return _info

In [126]:
info = get_to_from(df, symmetric_slack=0.2)
info

,diff,indicator,l_slack,u_slack,min_slack_ind
0,-0.25,0,-0.10,0.35,0
1,0.00,1,-0.10,0.10,0
2,0.15,1,-0.25,0.10,0
3,-0.10,0,-0.10,0.20,0
4,-0.10,0,-0.10,0.20,0
5,-0.30,0,-0.10,0.40,0
6,-0.10,0,-0.10,0.20,0
7,-0.10,0,-0.10,0.20,0
8,-0.10,0,-0.10,0.20,0
9,-0.20,0,-0.10,0.30,0


In [116]:
df

,from,to,label,real_start,real_end,diff
0,20.25,20.90,1,18.70,20.45,-0.25
1,22.20,22.60,1,20.40,22.40,0.00
2,24.15,24.70,1,22.35,24.50,0.15
3,28.35,28.85,1,26.65,28.55,-0.10
4,32.40,32.90,1,30.70,32.60,-0.10
5,36.15,36.85,1,34.65,36.35,-0.30
6,40.35,40.85,1,38.65,40.55,-0.10
7,44.30,44.80,1,42.60,44.50,-0.10
8,48.20,48.70,1,46.50,48.40,-0.10
9,52.30,52.90,1,50.70,52.50,-0.20
